In [1]:
# Importing the libraries
from DBN import DBN
import os
import torch
import copy
import torchvision
from torchvision import datasets,transforms
from torch.utils.data import Dataset,DataLoader
from powernets.gridinit import grid_init

import matplotlib
import pandapower.networks as nw  # for case name
import matplotlib.pyplot as plt
from prp_utils.dataloader import data_loader

import math
import numpy as np
from nerualnets.structnet import CustomNet, DBN_C

device = "cuda" if torch.cuda.is_available() else "cpu"

/home/huzuntao/PycharmProjects/DBN-AE/venv/lib/python3.8/site-packages/numba/core/errors.py:175: UserWarning: Insufficiently recent colorama version found. Numba requires colorama >= 0.3.9
  warnings.warn(msg)


In [6]:
case_new = nw.case14()
case_new.gen['vm_pu'] = 1.00
om, net = grid_init(case_name=case_new)
l_p = copy.deepcopy(case_new.load['p_mw'])
l_p = torch.as_tensor(np.concatenate([l_p.to_numpy(), np.zeros(11)]), dtype=torch.float32)

#
num_input, num_output = len(net.load) * 2, len(net.bus) * 2

path_current = os.getcwd()
path_PRP = path_current.rsplit("/", 2)[0]
# deal with the data
path_add_file = None
# _path_4 = '/home/huzuntao/PycharmProjects/Auto_model_Ps2Dl_v1/Power_Relia_Proj/data/bus4_64_1st_2022_8_29/15_9_22.npy'
_path_4 = path_PRP +'/' + 'data/bus_14_500_1st_2023_3_8/9_10_48.npy'
inps, outs = data_loader(the_path=_path_4, in_num=int(num_input / 2), out_num=int(num_output / 2))
inps = inps-l_p

# inpu_, labs_ = dataset_[:64 * 5]
# dataset = TensorDataset(inpu_, labs_)

In [3]:
path_current = os.getcwd()
path_current
path_PRP = path_current.rsplit("/", 2)[0]
path_PRP

'/home/huzuntao/PycharmProjects/DBN-AE'

In [7]:
inps[:1]


tensor([[ 2.5637, -3.3291,  5.0206,  0.7690,  0.4290,  2.6368,  1.2962,  0.5763,
         -0.6968, -2.2375, -0.4072, 12.7000, 19.0000, -3.9000,  1.6000,  7.5000,
         16.6000,  5.8000,  1.8000,  1.6000,  5.8000,  5.0000]])

In [8]:
inps[1:2]

tensor([[ 1.7856, 16.9898, -5.6308,  1.2520, -0.8544,  5.2334,  0.7907, -0.5432,
         -0.4318,  2.4507,  0.9800, 12.7000, 19.0000, -3.9000,  1.6000,  7.5000,
         16.6000,  5.8000,  1.8000,  1.6000,  5.8000,  5.0000]])

In [5]:
#Loading dataset and show the content
torch.unify_type_list()
for item in dataset_[:2]:
    print(f"Input: {item[0]}; \n Output: {item[1]}")
# a = dataset_[0]
# a[1]

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/huzuntao/PycharmProjects/DBN-AE/venv/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_27001/2842952277.py", line 2, in <module>
    torch.unify_type_list()
TypeError: unify_type_list(): incompatible function arguments. The following argument types are supported:
    1. (arg0: List[c10::Type]) -> c10::Type

Invoked with: 

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/huzuntao/PycharmProjects/DBN-AE/venv/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2057, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "/home/huzuntao/PycharmProjects/DBN-AE/venv/lib/python3.8/site-packages/IPython/core/ultratb.py", line 1288, in structured_traceback
    return FormattedTB.structured_traceback(
  File "/home/huzuntao/PycharmP

In [9]:
# I have have set these hyper parameters although you can experiment with them to find better hyperparameters.
nn_shape = [22, 23*23 ,23*23, 30*23, 18*18]
dbn_mnist = DBN(visible_units=22 ,
                hidden_units=[23*23 ,23*23, 30*23, 18*18] ,
                k = 5,
                learning_rate = 0.01,
                learning_rate_decay = True,
                xavier_init = True,
                increase_to_cd_k = False,
                use_gpu = False)

In [10]:
num_epochs = 20
batch_size = 20

dbn_mnist.train_static(inps,outs, num_epochs , batch_size)

--------------------
Training the 1 st rbm layer
RBM fitting: 25it [00:00, 404.73it/s]
Epoch:1 ,avg_cost = 9.550758177971133e+30 ,std_cost = 4.512942328856492e+31 ,avg_grad = 1.275900230906151e+18 ,std_grad = 4.96994813844285e+18
RBM fitting: 25it [00:00, 566.70it/s]
Epoch:2 ,avg_cost = inf ,std_cost = nan ,avg_grad = 1.7721971499053503e+24 ,std_grad = 4.097552779123763e+24
RBM fitting: 25it [00:00, 576.96it/s]
Epoch:3 ,avg_cost = inf ,std_cost = nan ,avg_grad = 4.930399639160905e+24 ,std_grad = 7.648938971399152e+24
RBM fitting: 25it [00:00, 528.56it/s]
Epoch:4 ,avg_cost = inf ,std_cost = nan ,avg_grad = 2.2246612630682017e+21 ,std_grad = 7.778659497640827e+21
RBM fitting: 25it [00:00, 580.74it/s]
Epoch:5 ,avg_cost = 8.730021245301043e+25 ,std_cost = 2.5724085491419623e+24 ,avg_grad = 0.0 ,std_grad = 0.0
RBM fitting: 25it [00:00, 530.44it/s]
Epoch:6 ,avg_cost = 7.961002310309795e+25 ,std_cost = 1.9545602206661622e+24 ,avg_grad = 0.0 ,std_grad = 0.0
RBM fitting: 25it [00:00, 560.73it/s

In [ ]:
dbn_mnist(inps[:1])

In [ ]:
dbn_mnist.state_dict()['W_rec%i'%1]

In [ ]:

nn_mlp = DBN_C([num_input, 123*23 ,23*23, 30*23, 18*18]).to(device)  # middle layer

# nn_mlp.load_state_dict(dbn_mnist.state_dict(), strict=False)

In [11]:
# copy the trained DBN for new NN
layers = len(nn_shape)-1
nn_mlp = DBN_C([num_input, 23*23 ,23*23, 30*23, 18*18])
for i in [1]:
    if i == layers-1:   # last layer
        nn_mlp.state_dict()['model.%i.weight'%2*i][:] = dbn_mnist.state_dict()['W_mem'][:]
        nn_mlp.state_dict()['model.%i.bias'%2*i][:] = dbn_mnist.state_dict()['h_bias_mem'][:]
    else:
        print(i)
        nn_mlp.state_dict()['model.%i.weight'%2*i][:] = dbn_mnist.state_dict()['W_rec%i'%i][:]
        nn_mlp.state_dict()['model.%i.bias'%2*i][:] = dbn_mnist.state_dict()['bias_rec%i'%i][:]
        print(nn_mlp.state_dict()['model.%i.bias'%2][:5])
        print(dbn_mnist.state_dict()['bias_rec%i'%i][:5])
i=1
# nn_mlp.state_dict()['model.%i.bias'%2*i][:]= dbn_mnist.state_dict()['bias_rec%i'%i][:]
print(nn_mlp.state_dict()['model.%i.weight'%2*i][:5])
# print(nn_mlp.state_dict()['model.%i.bias'%2*i])
print(dbn_mnist.state_dict()['W_rec%i'%i][:5])

1
tensor([0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0.])
tensor([[-0.0241,  0.0143, -0.0006,  ...,  0.0069,  0.0041, -0.0005],
        [-0.0216,  0.0283,  0.0095,  ..., -0.0193,  0.0060, -0.0138],
        [-0.0231,  0.0157, -0.0046,  ...,  0.0164,  0.0006,  0.0153],
        [-0.0025,  0.0032,  0.0163,  ..., -0.0252, -0.0103,  0.0136],
        [-0.0020, -0.0211, -0.0075,  ...,  0.0036, -0.0165, -0.0153]])
tensor([[-0.0241,  0.0143, -0.0006,  ...,  0.0069,  0.0041, -0.0005],
        [-0.0216,  0.0283,  0.0095,  ..., -0.0193,  0.0060, -0.0138],
        [-0.0231,  0.0157, -0.0046,  ...,  0.0164,  0.0006,  0.0153],
        [-0.0025,  0.0032,  0.0163,  ..., -0.0252, -0.0103,  0.0136],
        [-0.0020, -0.0211, -0.0075,  ...,  0.0036, -0.0165, -0.0153]])


In [12]:
nn_mlp.state_dict().keys()

odict_keys(['model.0.weight', 'model.0.bias', 'model.2.weight', 'model.2.bias', 'model.4.weight', 'model.4.bias', 'model.6.weight', 'model.6.bias'])

In [10]:
dbn_mnist(inps[:1])

(tensor([[0.2015, 0.3287, 0.3065, 0.4068, 0.2829, 0.4476, 0.1802, 0.4550, 0.6412,
          0.4885, 0.4545, 0.7448, 0.6926, 0.4626, 0.7484, 0.7320, 0.6925, 0.3099,
          0.3314, 0.4540, 0.4238, 0.6577, 0.3262, 0.4054, 0.4135, 0.4294, 0.3872,
          0.7419, 0.5482, 0.7144, 0.5732, 0.4287, 0.2338, 0.1624, 0.4013, 0.4472,
          0.5828, 0.5290, 0.3311, 0.3654, 0.4680, 0.3462, 0.4445, 0.5618, 0.3161,
          0.4289, 0.5324, 0.1796, 0.3753, 0.3440, 0.7034, 0.3720, 0.2386, 0.2060,
          0.4733, 0.4623, 0.3535, 0.2907, 0.4364, 0.3562, 0.2658, 0.3926, 0.5273,
          0.5016, 0.7128, 0.7961, 0.2156, 0.5733, 0.4009, 0.3968, 0.4339, 0.3952,
          0.3055, 0.5650, 0.4733, 0.5012, 0.4413, 0.4182, 0.6628, 0.2006, 0.4323,
          0.5046, 0.5472, 0.2793, 0.2289, 0.2568, 0.6460, 0.3905, 0.2508, 0.3728,
          0.3599, 0.2499, 0.3014, 0.5668, 0.6186, 0.2822, 0.4171, 0.2617, 0.5024,
          0.3024, 0.4374, 0.2079, 0.4935, 0.2748, 0.4304, 0.6037, 0.5785, 0.4493,
          0.3452

In [13]:
nn_mlp(inps[:1])

tensor([[0.4867, 0.5242, 0.6130, 0.4757, 0.4726, 0.4273, 0.3214, 0.5480, 0.5057,
         0.6445, 0.4526, 0.5080, 0.5879, 0.5883, 0.4232, 0.5231, 0.6153, 0.5006,
         0.5932, 0.4230, 0.5318, 0.5742, 0.4817, 0.4704, 0.4428, 0.4683, 0.6576,
         0.4905, 0.5205, 0.5115, 0.5447, 0.5071, 0.4491, 0.4875, 0.5651, 0.5039,
         0.5548, 0.5653, 0.5387, 0.4842, 0.5697, 0.6856, 0.6391, 0.5257, 0.5562,
         0.5731, 0.4544, 0.3489, 0.5345, 0.5043, 0.5300, 0.5466, 0.5327, 0.5265,
         0.4890, 0.4465, 0.4256, 0.5068, 0.5466, 0.5067, 0.4360, 0.5251, 0.4883,
         0.4477, 0.4644, 0.4918, 0.5755, 0.5688, 0.5718, 0.5959, 0.3609, 0.5468,
         0.6072, 0.5146, 0.6910, 0.4875, 0.4903, 0.5898, 0.4790, 0.4306, 0.4575,
         0.5434, 0.4116, 0.5068, 0.3955, 0.5275, 0.6195, 0.3692, 0.4907, 0.4699,
         0.4505, 0.6234, 0.4562, 0.6264, 0.4486, 0.5478, 0.5209, 0.4984, 0.4122,
         0.5874, 0.4661, 0.4411, 0.5096, 0.4057, 0.4755, 0.4950, 0.5384, 0.4617,
         0.5173, 0.5396, 0.4

In [ ]:
# visualising layer 1
learned_weights = dbn_mnist.rbm_layers[0].W.transpose(0,1).numpy()
plt.show()
fig = plt.figure(3, figsize=(10,10))
for i in range(25): 
    sub = fig.add_subplot(5, 5, i+1)
    sub.imshow(learned_weights[i,:].reshape((28,28)), cmap=plt.cm.gray)
plt.show()

In [ ]:
# visualising layer 2
learned_weights = dbn_mnist.rbm_layers[1].W.transpose(0,1).numpy()
plt.show()
fig = plt.figure(3, figsize=(10,10))
for i in range(25): 
    sub = fig.add_subplot(5, 5, i+1)
    sub.imshow(learned_weights[i,:].reshape((23,23)), cmap=plt.cm.gray)
plt.show()

In [ ]:
number = 5 #A number between 0 and 10.

particular_mnist = []

limit = mnist_data.data.shape[0]
# limit = 60000
for i in range(limit):
    if(mnist_data.targets[i] == number):
        particular_mnist.append(mnist_data.data[i].numpy())
# particular_mnist = np.array(particular_mnist)
len(particular_mnist)
# mnist_data.train_data

In [ ]:
train_data = torch.stack([torch.Tensor(i) for i in particular_mnist])
train_label = torch.stack([torch.Tensor(number) for i in range(len(particular_mnist))])

In [ ]:
dbn_mnist.train_static(train_data,train_label,30 , batch_size)

In [ ]:
idx = 3
img = mnist_data.train_data[idx]
reconstructed_img = img.view(1,-1).type(torch.FloatTensor)

_,reconstructed_img= dbn_mnist.reconstruct(reconstructed_img)

reconstructed_img = reconstructed_img.view((28,28))
print("The original number: {}".format(mnist_data.train_labels[idx]))
plt.imshow(img , cmap = 'gray')
plt.show()
print("The reconstructed image")
plt.imshow(reconstructed_img , cmap = 'gray')
plt.show()